https://henryfeng.medium.com/expand-twitter-follower-network-for-ngo-with-python-tweepy-package-a731e11195f0


https://stackoverflow.com/questions/31000178/how-to-get-large-list-of-followers-tweepy

https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/user

## Generate the list of accounts’ followers

In [1]:
import tweepy
import json
import time
import pandas as pd
import networkx as nx
from pyvis.network import Network

In [2]:
tweepy.__version__

'4.8.0'

In [3]:
C_KEY = ''
C_SECRET = ''
A_TOKEN = ''
A_TOKEN_SECRET = ''

In [4]:
auth = tweepy.OAuthHandler(C_KEY, C_SECRET)
auth.set_access_token(A_TOKEN, A_TOKEN_SECRET)

In [5]:
account1 = "AaltoUniversity"
account2 = "helsinkiuni"

In [6]:
"AaltoUniversity", "helsinkiuni"

('AaltoUniversity', 'helsinkiuni')

In [7]:
# Set wait_on_rate_limit to True while connecting through Tweepy
api = tweepy.API(auth, wait_on_rate_limit=True)

In [32]:
def get_userids(account, id_type = 'user_id'):
    # Get user ids as a list
    # request for 5000 follower ids in one request --> 75K ids in every 15 minute window
    print(account, id_type)
    userids =[]
    if id_type=='screen_name':
        for user in tweepy.Cursor(api.get_follower_ids, screen_name=account, count=5000).items():
            userids.append(user)  
    elif id_type=='user_id':
        for user in tweepy.Cursor(api.get_follower_ids, user_id=account, count=5000).items():
            userids.append(user) 
    return userids

In [35]:
api.get_user(screen_name="helsinkiuni")

User(_api=<tweepy.api.API object at 0x7f954d413950>, _json={'id': 168607383, 'id_str': '168607383', 'name': 'University of Helsinki', 'screen_name': 'helsinkiuni', 'location': 'Helsinki, Finland', 'profile_location': None, 'description': 'University of Helsinki - Helsingin yliopisto is Finland’s largest, oldest and internationally most esteemed research university. #HelsinkiUni #WeAreHelsinkiUni', 'url': 'https://t.co/mZODiLVf4n', 'entities': {'url': {'urls': [{'url': 'https://t.co/mZODiLVf4n', 'expanded_url': 'http://www.helsinki.fi', 'display_url': 'helsinki.fi', 'indices': [0, 23]}]}, 'description': {'urls': []}}, 'protected': False, 'followers_count': 53437, 'friends_count': 2467, 'listed_count': 437, 'created_at': 'Tue Jul 20 10:12:30 +0000 2010', 'favourites_count': 31974, 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'verified': True, 'statuses_count': 18686, 'lang': None, 'status': {'created_at': 'Tue May 03 16:00:01 +0000 2022', 'id': 1521519960522174467, 'id_str

In [10]:
def get_userids(account, api):
    # Get user ids
    # request for 5000 follower ids in one request --> 75K ids in every 15 minute window
    
    userids =[]
    for user in tweepy.Cursor(api.get_follower_ids, screen_name=account, count=5000).items():
        userids.append(user) 
    return userids
    
def get_usernames(userids, api):
    
    # The limit for the users/lookup endpoint is 90000 users per 15 minute window 
    # (900 requests x 100 users per request)
    fullusers = []
    u_count = len(userids)
    print(u_count)
    try:
        for i in range(int(u_count/100) + 1):            
            end_loc = min((i + 1) * 100, u_count)
            fullusers.extend(
                api.lookup_users(user_id=userids[i * 100:end_loc])                
            )
        return fullusers
    except:
        import traceback
        traceback.print_exc()
        print ('Something went wrong, quitting...')

In [8]:
%%time

# # fetch ids of account followers

# userids_Aalto  = get_userids(account1, api)
# userids_UniHel = get_userids(account2, api)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.2 µs


In [9]:
# # create a df 

# userids = list(set(userids_Aalto + userids_UniHel))

# df = pd.DataFrame(userids, columns=['userids'])
# df.head()


# df['Aalto'] = 0
# df['UniHel'] = 0

# df['Aalto'] = df['userids'].isin(userids_Aalto).astype(int)
# df['UniHel'] = df['userids'].isin(userids_UniHel).astype(int)
# df.head()

In [10]:
# df.to_csv("userids.csv", sep='\t', index=False)

In [17]:
# read in saved df

df = pd.read_csv("userids.csv", sep='\t')
df.head()

,Unnamed: 0,userids,Aalto,UniHel
0,0,1294684866747957248,1,1
1,1,950777123769286656,1,0
2,2,714929181289742336,1,1
3,3,785480415813435392,1,1
4,4,923477420828655616,1,0


In [ ]:
# get followers of each user

for i in range(6,7):
    followers = {}
    for user in userids[i*100:i*100+100]:
        print(user)
        followers_id = api.get_follower_ids(user_id=user)
        followers[user]=followers_id
    # save followers
    
    name = 'followers' + str(i+1) + 'json'
    with open('followers2.json', 'w') as fp:
        json.dump(followers, fp)

In [ ]:
# get followers of each user

for i in range(5,6):
    followers = {}
    for user in userids[i*100:i*100+100]:
        print(user)
        try:
            followers_id = api.get_follower_ids(user_id=user)
            followers[user]=followers_id
        except:
            pass
    # save followers
    
    name = 'followers' + str(i+1) + 'json'
    with open('followers2.json', 'w') as fp:
        json.dump(followers, fp)

In [ ]:
# # save followers

# with open('followers2.json', 'w') as fp:
#     json.dump(followers, fp)

In [13]:
# open JSON file

followers = {}
for i in range(1,7):
    name = 'followers' + str(i) + '.json'
    with open(name) as json_file:
        f = json.load(json_file)
    followers.update(f)

In [14]:
len(followers)

445

In [15]:
userids = list(df['userids'])
len(userids)

78363

In [16]:
nodes = list(followers.keys())
print("Number of nodes: ", len(nodes))

# assign colors
color_map = []

for node in nodes:
    if (df[df['userids']==int(node)].Aalto==1).values and (df[df['userids']==int(node)].UniHel==1).values:
        color_map.append('#D85678')
    elif (df[df['userids']==int(node)].Aalto==1).values:
        color_map.append('#FFA500')
    else:
        color_map.append('#6495ed')

445


In [54]:
G = nx.DiGraph()

for i, node in enumerate(nodes):
    G.add_node(node, color=color_map[i])

In [55]:
for node in nodes:
    # nodes is a list of strings
    # followers[node] returns list of ints
    for follower in followers[node]:
        if str(follower) in followers:
            # iteration of followers returns keys, which are strings
            # nodes in G are str
            G.add_edge(node, str(follower))

In [56]:
G.number_of_nodes(), G.number_of_edges()

(445, 206)

In [57]:
G.remove_nodes_from(list(nx.isolates(G)))

In [58]:
G.number_of_nodes(), G.number_of_edges()

(122, 206)

### Add nodes for account 1 and 2 (those which we are comparing)

In [79]:
net = Network(height='800px', width='100%', directed=True, notebook=True)

net.add_node(account1, size=20, color='#f54242', label="Aalto", font='30px arial black')
net.add_node(account2, size=20, color='#427ef5', label="Uni of Helsinki", font='30px arial black')

for n in G.nodes:
    color = G.nodes[n]['color']
    net.add_node(n, size=20, color=color)
    if G.nodes[n]['color'] == '#D85678':
        net.add_edge(n, account1)
        net.add_edge(n, account2)
    elif  G.nodes[n]['color'] == '#FFA500':
        net.add_edge(n, account1)
    else:
        net.add_edge(n, account2)
        
for n1,n2, attrs in G.edges.data():
    net.add_edge(n1, n2)

In [80]:
net.toggle_physics(True)
# net.show_buttons()
# net.show('mygraph.html')

net.show_buttons(filter_=['physics'])
net.show('graph.html')

In [ ]:
nodes = list(G.nodes)

In [ ]:
len(nodes), type(nodes[0]), nodes[0]

In [ ]:
# get info about each user

img = []
screen_name = []
for node in nodes:
    info = api.lookup_users(user_id=[int(node)])
    img.append(info[0].profile_image_url_https)
    screen_name.append(info[0].screen_name)

In [ ]:
len(img), len(screen_name)

In [ ]:
G = nx.DiGraph()

for i, node in enumerate(nodes):
    G.add_node(node, color=color_map[i], image=img[i], label=screen_name[i])

In [ ]:
for node in nodes:
    # nodes is a list of strings
    # followers[node] returns list of ints
    for follower in followers[node]:
        if str(follower) in followers:
            # iteration of followers returns keys, which are strings
            # nodes in G are str
            G.add_edge(node, str(follower))

In [ ]:
G.number_of_nodes(), G.number_of_edges()

In [ ]:
net = Network(height='800px', width='100%', directed=True, notebook=True)

for i, node in enumerate(G.nodes):
    net.add_node(node, color=color_map[i], image=img[i], label=screen_name[i], shape='image')
    
# for n in G.nodes:
#     color = G.nodes[n]['color']
#     net.add_node(n, color=color)
for n1,n2, attrs in G.edges.data():
    net.add_edge(n1, n2)

In [ ]:
net.toggle_physics(True)
# net.show_buttons()
# net.show('mygraph.html')

# net.show_buttons(filter_=['physics'])
options = """
var options = {
  "configure": {
        "enabled": true
   },
  "physics": {
    "springLength": 100
  }
}
"""
net.set_options(options)
net.show('graph_img.html')